<a href="https://colab.research.google.com/github/shinnew9/AICounselors/blob/main/data/data_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd

/content


In [5]:
%cd /content/drive/MyDrive/ProfChuah

/content/drive/MyDrive/ProfChuah


In [11]:
!pwd

/content/drive/MyDrive/ProfChuah


In [14]:
file_path = "./PsyDial4_en_only.txt"

with open(file_path, "r", encoding="utf-8") as f:
  lines = f.readlines()

for line in lines:
  print(line)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[user] Well, yes. Every time I talk to someone, I feel a little more than empty and helpless. Especially those who really know how I feel, their words are a great comfort to me.

[assistant] You mentioned going to the hospital, is it a feeling that we need to adjust further?

[user] Well, it's not worse, it's just a little up and down the previous period. The doctor suggested I review it, confirm the latest state, adjust the medication.

[assistant] So, what's the special behavior you've been doing lately when you've been depressed?

[user] Yes, sometimes when I'm depressed, I can't help betting, as if only in that extreme stimulus would I find some short pleasure and forget.

[assistant] I know this feeling may help you out of your pain for a short time. Do you have another way of exhaleing?

[user] Well, I'll be careful. If I need to, I'll go see the doctor and adjust the situation.

[assistant] Good, your positive attitude is important. You can ca

In [16]:
# To DF

import re
import pandas as pd

path = "/content/drive/MyDrive/ProfChuah/PsyDial4/en_translated/PsyDial4_en_only.txt"

session_header = re.compile(r"^=== SESSION (\d+) ===\s*$")
turn_line = re.compile(r"^\[(system|user|assistant)\]\s*(.*)$")

rows = []
current_session = None
turn_id = 0

with open(path, "r", encoding="utf-8") as f:
  for raw in f:
    line = raw.rstrip("\n")

    m = session_header.match(line)
    if m:
      current_session = int(m.group(1))
      turn_id = 0
      continue

    m = turn_line.match(line)
    if m and current_session is not None:
      role = m.group(1)
      text = m.group(2).strip()
      rows.append({
          "session_id": current_session,
          "turn_id": turn_id,
          "role": role,
          "text": text
      })
      turn_id += 1

df = pd.DataFrame(rows)
df.head(10), df["session_id"].nunique(), len(df)

(   session_id  turn_id       role  \
 0           0        0     system   
 1           0        1       user   
 2           0        2  assistant   
 3           0        3       user   
 4           0        4  assistant   
 5           0        5       user   
 6           0        6  assistant   
 7           0        7       user   
 8           0        8  assistant   
 9           0        9       user   
 
                                                 text  
 0  The following is a message from the daytime da...  
 1                                             Hello.  
 2  Please fill out the evaluation questionnaire b...  
 3                                              Okay.  
 4  Okay, thank you. Today we can talk about your ...  
 5  I wonder why I'm so confused and empty now. Af...  
 6  I see. Did you just mention confusion and empt...  
 7  It's been a week since I went back to school, ...  
 8  Sounds like you've been through a lot of confu...  
 9  Yes. I always th

In [18]:
df.tail(10)

,session_id,turn_id,role,text
182438,2381,55,user,"It's probably a habit to be older, to be compa..."
182439,2381,56,assistant,What exactly is lack of talent?
182440,2381,57,user,I felt that I had no particular merit and no s...
182441,2381,58,assistant,You think it's hard to accept without a partic...
182442,2381,59,user,"It seems like that, I always felt I had to be ..."
182443,2381,60,assistant,I understand how you feel. We live in a societ...
182444,2381,61,user,"Yeah, I know that, but it's really hard to do."
182445,2381,62,assistant,"I understand the contradiction of the feeling,..."
182446,2381,63,user,"Thank you, that makes me feel a little relieved."
182447,2381,64,assistant,You're welcome. It's a great development to fe...


In [17]:
df.to_csv("/content/drive/MyDrive/ProfChuah/PsyDial4/en_translated/PsyDial4.csv", index=False)

In [25]:
# To JSON

import re, json

path = "/content/drive/MyDrive/ProfChuah/PsyDial4/en_translated/PsyDial4_en_only.txt"
out_path = "/content/drive/MyDrive/ProfChuah/PsyDial4/en_translated/PsyDial4_sessions.json"

session_header = re.compile(r"^=== SESSION (\d+) ===\s*$")
turn_line = re.compile(r"^\[(system|user|assistant)\]\s*(.*)$")

sessions = []
cur = None

with open(path, "r", encoding="utf-8") as f:
  for raw in f:
    line = raw.rstrip("\n")

    m = session_header.match(line)
    if m:
      if cur is not None:
        sessions.append(cur)
      cur = {"session_id": int(m.group(1)), "turns": []}
      continue

    m = turn_line.match(line)
    if m and cur is not None:
      cur["turns"].append({"role":m.group(1), "text":m.group(2).strip()})


# the last session flush
if cur is not None:
  sessions.append(cur)

with open(out_path, "w", encoding="utf-8") as w:
  for s in sessions:
    w.write(json.dumps(s, ensure_ascii=False) + "\n")

print("num_sessions:", len(sessions))
print("saved:", out_path)
print("example:", sessions[0]["turns"][:3])

num_sessions: 2382
saved: /content/drive/MyDrive/ProfChuah/PsyDial4/en_translated/PsyDial4_sessions.json
example: [{'role': 'system', 'text': 'The following is a message from the daytime day: "The role name: The role name: The sex: The role of the woman: Virtual psychologist, good at humanism, psychoanalytic and cognitive behavioral therapy. Skills: Helping to identify and challenge unhealthy thinking, providing psychological support and empathy.'}, {'role': 'user', 'text': 'Hello.'}, {'role': 'assistant', 'text': 'Please fill out the evaluation questionnaire before we start officially.'}]


In [28]:
# sessions[0]
# sessions[1]

In [35]:
import re, json, random, os
import pandas as pd

random.seed(42)
sampled_session = random.sample(sessions, 100)
print("\n sampled 100 sessions. first sampled id:", sampled_session[0]["session_id"])


 sampled 100 sessions. first sampled id: 456


In [39]:
# random sample 100

print("Total sessions:", len(sessions))

# 1) reproducible random sampling
random.seed(42)
sampled_sessions = random.sample(sessions, 100)

print("Sampled sessions:", len(sampled_sessions))
print("First 10 sampled session_ids:", [s["session_id"] for s in sampled_sessions[:10]])


# 2) save sampled sessions to jsonl
out_path = "/content/drive/MyDrive/ProfChuah/PsyDial4/en_translated/sample100_sessions.jsonl"
with open(out_path, "w", encoding="utf-8") as f:
  for s in sampled_sessions:
    f.write(json.dumps(s, ensure_ascii=False) + "\n")

  print("Saved to:", out_path)

Total sessions: 2382
Sampled sessions: 100
First 10 sampled session_ids: [456, 102, 1126, 1003, 914, 571, 419, 2233, 356, 1728]
Saved to: /content/drive/MyDrive/ProfChuah/PsyDial4/en_translated/sample100_sessions.jsonl


In [ ]:
#